In [17]:
import os
import librosa
import numpy as np
import pandas as pd

In [18]:
# Base directory is current folder 
base_dir = "."

# Emotion code to label map
emotion_map = {
    "01": "neutral",
    "02": "calm",
    "03": "happy",
    "04": "sad",
    "05": "angry",
    "06": "fearful",
    "07": "disgust",
    "08": "surprised"
}

In [19]:
def extract_label(filename):
    parts = filename.split("-")
    emotion_code = parts[2]
    return emotion_map.get(emotion_code, "unknown")

def extract_features(file_path):
    y, sr = librosa.load(file_path, sr=22050)
    
    # MFCCs
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
    mfcc_mean = np.mean(mfcc.T, axis=0)

    # Chroma
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    chroma_mean = np.mean(chroma.T, axis=0)

    # Spectral contrast
    contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
    contrast_mean = np.mean(contrast.T, axis=0)

    # Zero-crossing rate
    zcr = librosa.feature.zero_crossing_rate(y)
    zcr_mean = np.mean(zcr.T, axis=0)

    # Concatenate all
    return np.hstack((mfcc_mean, chroma_mean, contrast_mean, zcr_mean))



In [20]:

features = []
labels = []

for folder in ["speech", "song"]:
    folder_path = os.path.join(base_dir, folder)
    print(f"Processing folder: {folder_path}")
    
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".wav"):
            file_path = os.path.join(folder_path, file_name)
            try:
                label = extract_label(file_name)
                mfcc_features = extract_features(file_path)
                features.append(mfcc_features)
                labels.append(label)
                print(f"Processed: {file_name} -> Label: {label}")
            except Exception as e:
                print(f"Error processing {file_name}: {e}")

df = pd.DataFrame(features)
df["label"] = labels
df.to_csv("ravdess_features.csv", index=False)
print("✅ Features and labels saved to ravdess_features.csv")
import os

speech_files = os.listdir("./speech")
song_files = os.listdir("./song")

print("Speech folder:", len([f for f in speech_files if f.endswith(".wav")]), "files")
print("Song folder:", len([f for f in song_files if f.endswith(".wav")]), "files")
       

Processing folder: ./speech
Processed: 03-01-06-01-02-02-02.wav -> Label: fearful
Processed: 03-01-05-01-02-01-16.wav -> Label: angry
Processed: 03-01-08-01-01-01-14.wav -> Label: surprised
Processed: 03-01-06-01-02-02-16.wav -> Label: fearful
Processed: 03-01-05-01-02-01-02.wav -> Label: angry
Processed: 03-01-01-01-02-02-06.wav -> Label: neutral
Processed: 03-01-02-01-02-01-12.wav -> Label: calm
Processed: 03-01-01-01-02-02-12.wav -> Label: neutral
Processed: 03-01-02-01-02-01-06.wav -> Label: calm
Processed: 03-01-02-02-01-01-06.wav -> Label: calm
Processed: 03-01-02-02-01-01-12.wav -> Label: calm
Processed: 03-01-06-02-01-02-16.wav -> Label: fearful
Processed: 03-01-05-02-01-01-02.wav -> Label: angry
Processed: 03-01-08-02-02-01-14.wav -> Label: surprised
Processed: 03-01-06-02-01-02-02.wav -> Label: fearful
Processed: 03-01-05-02-01-01-16.wav -> Label: angry
Processed: 03-01-05-01-01-01-22.wav -> Label: angry
Processed: 03-01-08-01-02-01-20.wav -> Label: surprised
Processed: 03-01